# Problem
A matching in a graph is a **noncrossing** if none of its edges cross each other. If we assume that the $n$ nodes of this graph are arranged around a circle, and if we label these nodes with positive integers between 1 and n, then a matching is noncrossing as long as there are not edges {i, j} and {k, l} such that i < k < j < l

A noncrossing matching of basepair edges in the bounding graph corresponding to an RNA string will correspond to a possible secondary structure of the underlying RNA strand that lacks pseudoknots.

In this problem, we will consider counting noncrossing perfect matchings of basepair edges. As a motivating example of how to count noncrossing perfect matchings, let $c_n$ denote the number of noncrossing perfect matchings in the complete graph $K_{2n}$. After setting $c_0 = 1$, we can see that $c_1$ should equal 1 as well. As for the case of a general n, say that the nodes of $K_{2n}$ are labeled with the positive integers from 1 to 2n. We can join node 1 to any of remaining 2n - 1 nodes; yet once we have chosen this node(say m), we cannot add another edge to the matching that crosses the edge {1, m}. As a result, we must match all the edges on one side of {1, m} to each other. This requirement forces m to be even, so that we can write m = 2k for some positive integer k.

There are 2k-2 nodes on one side of {1, m} and 2n - 2k nodes on the other side of {1, m}, so that in turn there will be $c_{k-1} \cdot c_{n-k}$ different ways of forming a perfect matching on the remaining nodes of $K_{2n}$. If we let m vary over all possible n-1 choices of even numbers between 1 and 2n, then we obtain the recurrence relation $c_n = \sum^n_{k=1}c_{k-1} \cdot c_{n-k}$. The resulting numbers $c_n$ counting noncrossing perfect matchings in $K_{2n}$ are called the Catalan numbers, and they appear in a huge number of other settings.

**Given** : An RNA string s having the same number of occurrences of 'A' as 'U' and the same number of occurrences of 'C' as 'G'. The length of the string is at most 300bp

**Return** : The total number of noncrossing perfect matchings of basepair edges in the bonding graph of s, modulo 1,000,000

In [1]:
import toolz as tz
from toolz.curried import *
from operator import *
from functools import lru_cache

In [3]:
input_data = {
    "sample": {
        "fasta": """>Rosalind_57
AUCGAUCGCG"""
    },
    "test": {
        "fasta": open("data/rosalind_cat.txt", "r").read()
    }
}

cur_state = "test"
cur_data = input_data[cur_state]

input_processor = compose(
    list,
    map(lambda x: {
        "nt": x[0], "order": x[1] 
    }),
    lambda x: zip(x, range(len(x))),
    reduce(add),
    filter(lambda x: not x.startswith(">") and tz.identity(x)),
    flip(str.split, "\n"))

input_processor(cur_data["fasta"])

def match(a, b):
    if a["nt"] == "A" and b["nt"] == "U": return True
    elif a["nt"] == "C" and b["nt"] == "G": return True
    elif a["nt"] == "U" and b["nt"] == "A": return True
    elif a["nt"] == "G" and b["nt"] == "C": return True
    else: return False

@tz.curry
def meq(nt, x): return nt == x["nt"]


def tree(seq, mem={}):
    # print(seq)
    if len(seq) == 0: 
        return 1
    
    if len(seq) == 2 and match(first(seq), last(seq)):
        return 1
    s = list(map(get_in(["nt"]), seq))
    if s.count("A") != s.count("U"):
        return 0
    
    if s.count("C") != s.count("G"):
        return 0

    if ''.join(s) in mem.keys():
        # print("in")
        return mem[''.join(s)]

    start = first(seq)
    cnt = 0
    for target_idx in range(1, len(seq)):
        if match(start, seq[target_idx]):
            left = tree(seq[1:target_idx], mem=mem)
            right = tree(seq[target_idx + 1:], mem=mem)
            left_str = ''.join(list(map(get_in(["nt"]), seq[1:target_idx])))
            right_str = ''.join(list(map(get_in(["nt"]), seq[target_idx+1:])))
            mem[left_str] = left
            mem[right_str] = right
            cnt += left * right
            # print("left : {} {} | right {} {}".format(left_str, left, right_str, right))
    return cnt

run = compose(tree, input_processor)

print("result :", run(cur_data["fasta"]) % 1_000_000)

result : 88256


In [ ]:
''.join([])

''